<a href="https://colab.research.google.com/github/offthemenu/data-feature-ywc261/blob/main/data_features_jacketnojacket.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
# Set up API keys

from google.colab import userdata

ACCU_API_KEY = userdata.get("ACCU_API_KEY")

IP_API_KEY = userdata.get("IPSTACK_API_KEY")

BD_API_KEY = userdata.get("BD_API_KEY")



if ACCU_API_KEY:
    print("Successfully loaded Accuweather API Key")
else:
    print("Failed to load Accuweather API Key")

if IP_API_KEY:
    print("Successfully loaded IP Stack API Key")
else:
    print("Failed to load IP Stack API Key")

if BD_API_KEY:
    print("Successfully loaded new IP Stack API Key")
else:
    print("Failed to load new IP Stack API Key")

Successfully loaded Accuweather API Key
Successfully loaded IP Stack API Key
Successfully loaded new IP Stack API Key


## 1. Create a function that will return a public IP address for the user.

In [21]:
import requests

def getIP():
  url_getip = f"https://api-bdc.net/data/client-ip"
  r_getip = requests.get(url_getip)
  data_getip = r_getip.json()
  ip_address = data_getip['ipString']
  # ip_type = data_getip["ipType"]
  # print(ip_address, ip_type)

  return ip_address

# Test results and store the returned output to the ip_address variable
ip_address = getIP()
print(ip_address)

34.118.194.141


## 2. Create a function that will return the corresponding address.

In [22]:
import requests

def get_location(ipAddress):
  url_ip = f"https://api-bdc.net/data/ip-geolocation?ip={ip_address}&localityLanguage=en&key={BD_API_KEY}"
  r_ip = requests.get(url_ip)
  data_ip = r_ip.json()
  data_ip
  location_query = f"{data_ip['location']['localityName']}, {data_ip['location']['principalSubdivision']}"
  return location_query

# Test results and store the returned output to the location_query variable
location_query = get_location(ip_address)
print(location_query)

The Dalles, Oregon


## 3. Create a function that will return 4 varialbes: weather, temp in C, temp in F, and precipitation from the name of the location

In [23]:
url_weather_loc = f"http://dataservice.accuweather.com/locations/v1/cities/search?apikey={ACCU_API_KEY}&q={location_query}"
r_weather_loc = requests.get(url_weather_loc)
data_weather_loc = r_weather_loc.json()
data_weather_loc
location_key = data_weather_loc[0]['Key']

In [24]:
def get_weather(location_query):
  '''
  location query: string in the format of "city, state"
  output: (in the order of) current weather, current temperature in Celsius, current temperature in Fahrenheit, and precipitation.
  '''

  url_weather_loc = f"http://dataservice.accuweather.com/locations/v1/cities/search?apikey={ACCU_API_KEY}&q={location_query}"
  r_weather_loc = requests.get(url_weather_loc)
  data_weather_loc = r_weather_loc.json()
  location_key = data_weather_loc[0]['Key']

  url_currentweather = f"http://dataservice.accuweather.com/currentconditions/v1/{location_key}?apikey={ACCU_API_KEY}"
  r_currentweather = requests.get(url_currentweather)
  data_currentweather = r_currentweather.json()
  current_weather = data_currentweather[0]['WeatherText']
  current_temp_F = data_currentweather[0]['Temperature']['Imperial']['Value']
  current_temp_C = data_currentweather[0]['Temperature']['Metric']['Value']
  precipitation = data_currentweather[0]['HasPrecipitation']

  return current_weather, current_temp_C, current_temp_F, precipitation

# Test results and store the returned output to the respective variables

current_weather, current_tempC, current_tempF, precipitation = get_weather(location_query)
print(current_weather, current_tempC, current_tempF, precipitation)

Mostly cloudy 16.7 62.0 False


## 4. Combine all the functions above into a single program

In [25]:
userInput = input("Would you like to know if you'll need a jacket? (Yes/No)").lower()

if userInput == "yes":
    ipAddress = getIP()

    currentLocation = get_location(ipAddress)

    current_weather, current_tempC, current_tempF, precipitation = get_weather(currentLocation)

    if current_tempC < 15:
      if precipitation == True:
        print(f"The weather in {currentLocation} is currently {current_weather} with the temperature at {current_tempC} Celsius or {current_tempF} Farenheit.\nYou might want to consider bringing a light jacket.\nAlso, it is currently raining outside–you might want to bring an umbrella.")
      else:
        print(f"The weather in {currentLocation} is currently {current_weather} with the temperature at {current_tempC} Celsius or {current_tempF} Farenheit.\nYou might want to consider bringing a light jacket.")
    elif current_tempC < 5:
      if precipitation == True:
        print(f"The weather in {currentLocation} is currently {current_weather} with the temperature at {current_tempC} Celsius or {current_tempF} Farenheit.\nYou'll want to bring a jacket outside.\nAlso, it is currently raining outside–you might want to bring an umbrella.")
      else:
        print(f"The weather in {currentLocation} is currently {current_weather} with the temperature at {current_tempC} Celsius or {current_tempF} Farenheit.\nYou'll want to bring a jacket outside.")
    else:
      if precipitation == True:
        print(f"The weather in {currentLocation} is currently {current_weather} with the temperature at {current_tempC} Celsius or {current_tempF} Farenheit.\nNo need to bring a jacket.\nHowever, it is currently raining outside–you might want to bring an umbrella.")
      else:
        print(f"The weather in {currentLocation} is currently {current_weather} with the temperature at {current_tempC} Celsius or {current_tempF} Farenheit.\nNo need to bring a jacket.")
else:
    print("Okay, bye!")

Would you like to know if you'll need a jacket? (Yes/No)yes
The weather in The Dalles, Oregon is currently Mostly cloudy with the temperature at 16.7 Celsius or 62.0 Farenheit.
No need to bring a jacket.
